In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Input
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.datasets.citation import Cora
from spektral.layers import GATConv
from spektral.transforms import AdjToSpTensor, LayerPreprocess
from spektral.utils import tic, toc

tf.random.set_seed(0)

# Load data
dataset = Cora(normalize_x=True, transforms=[LayerPreprocess(GATConv), AdjToSpTensor()])
graph = dataset[0]
x, a, y = graph.x, graph.a, graph.y
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

l2_reg = 2.5e-4
# Define model
x_in = Input(shape=(dataset.n_node_features,))
a_in = Input(shape=(None,), sparse=True)

x_1 = Dropout(0.6)(x_in)
print('shape of x1 -->', x_1.shape)
x_1 = GATConv(
    8,
    attn_heads=8,
    concat_heads=True,
    dropout_rate=0.6,
    activation="elu",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([x_1, a_in])
print('shape of x1 after GATconv -->', x_1.shape)
x_2 = Dropout(0.6)(x_1)
x_2 = GATConv(
    dataset.n_labels,
    attn_heads=1,
    concat_heads=False,
    dropout_rate=0.6,
    activation="softmax",
    kernel_regularizer=l2(l2_reg),
    attn_kernel_regularizer=l2(l2_reg),
    bias_regularizer=l2(l2_reg),
)([x_2, a_in])

# Build model
model = Model(inputs=[x_in, a_in], outputs=x_2)
optimizer = Adam(learning_rate=5e-3)
loss_fn = CategoricalCrossentropy()


# Training step
@tf.function
def train():
    with tf.GradientTape() as tape:
        predictions = model([x, a], training=True)
        loss = loss_fn(y[mask_tr], predictions[mask_tr])
        loss += sum(model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


@tf.function
def evaluate():
    predictions = model([x, a], training=False)
    losses = []
    accuracies = []
    for mask in [mask_tr, mask_va, mask_te]:
        loss = loss_fn(y[mask], predictions[mask])
        loss += sum(model.losses)
        losses.append(loss)
        acc = tf.reduce_mean(categorical_accuracy(y[mask], predictions[mask]))
        accuracies.append(acc)
    return losses, accuracies


best_val_loss = 99999
best_test_acc = 0
current_patience = patience = 100
epochs = 999999
tic()
for epoch in range(1, epochs + 1):
    train()
    l, a = evaluate()
    print(
        "Loss tr: {:.4f}, Acc tr: {:.4f}, "
        "Loss va: {:.4f}, Acc va: {:.4f}, "
        "Loss te: {:.4f}, Acc te: {:.4f}".format(l[0], a[0], l[1], a[1], l[2], a[2])
    )
    if l[1] < best_val_loss:
        best_val_loss = l[1]
        best_test_acc = a[2]
        current_patience = patience
        print("Improved")
    else:
        current_patience -= 1
        if current_patience == 0:
            print("Test accuracy: {}".format(best_test_acc))
            break
toc("GAT ({} epochs)".format(epoch))

Pre-processing node features
shape of x1 --> (None, 1433)
shape of x1 after GATconv --> (None, 32)


KeyboardInterrupt: 

In [10]:
dataset[0]

Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)

In [11]:
# no of nodes : 2708
# no of features : 1433

In [15]:
x.shape

(2708, 1433)

In [16]:
x_in.shape

TensorShape([None, 1433])

In [17]:
a_in.shape

TensorShape([None, None])

In [5]:
x_1.shape

TensorShape([None, 64])

In [6]:
a_in

<KerasTensor: type_spec=SparseTensorSpec(TensorShape([None, None]), tf.float32) (created by layer 'input_2')>

In [12]:
dataset.n_node_features

1433

In [21]:
x_2.shape

TensorShape([None, 7])

In [22]:
dataset.n_labels

7

In [24]:
a.shape

TensorShape([2708, 2708])

In [3]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import BatchLoader
from spektral.datasets import TUDataset
from spektral.layers import GCSConv, GlobalSumPool, GraphMasking, MinCutPool

import tensorflow.keras.backend as K
import tensorflow as tf

################################################################################
# Config
################################################################################
learning_rate = 1e-3  # Learning rate
epochs = 10  # Number of training epochs
batch_size = 32  # Batch size

################################################################################
# Load data
################################################################################
dataset = TUDataset("PROTEINS", clean=True)

# Parameters
N = max(g.n_nodes for g in dataset)
F = dataset.n_node_features  # Dimension of node features
S = dataset.n_edge_features  # Dimension of edge features
n_out = dataset.n_labels  # Dimension of the target

# Train/test split
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]

loader_tr = BatchLoader(dataset_tr, batch_size=batch_size, mask=True)
loader_va = BatchLoader(dataset_va, batch_size=batch_size, mask=True)
loader_te = BatchLoader(dataset_te, batch_size=batch_size, mask=True)


################################################################################
# Build model
################################################################################
class Net(Model):
    def __init__(self):
        super().__init__()
        self.mask = GraphMasking()
        self.conv1 = GCSConv(32, activation="relu")
        self.pool = MinCutPool(N // 2)
        self.conv2 = GCSConv(32, activation="relu")
        self.global_pool = GlobalSumPool()
        self.dense1 = Dense(n_out)

    def call(self, inputs):
        x, a = inputs
        print('shape of x --> ', x.shape, K.eval(tf.rank(x)) )
        print('shape of a --> ', a.shape, K.eval(tf.rank(a)) )
        x = self.mask(x)
        x = self.conv1([x, a])
        x_pool, a_pool = self.pool([x, a])
        x_pool = self.conv2([x_pool, a_pool])
        output = self.global_pool(x_pool)
        output = self.dense1(output)

        return output


model = Net()
opt = Adam(learning_rate=learning_rate)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["acc"])

################################################################################
# Fit model
################################################################################
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    epochs=epochs,
    validation_data=loader_va,
    validation_steps=loader_va.steps_per_epoch,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
)

################################################################################
# Evaluate model
################################################################################
print("Testing model")
loss, acc = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}. Test acc: {}".format(loss, acc))

C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Successfully loaded PROTEINS.
shape of x -->  (32, 93, 5) 3
shape of a -->  (32, 93, 93) 3
Epoch 1/10


C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\spektral\data\utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)
C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


AttributeError: in user code:

    File "C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Sikdara\.conda\envs\fgenv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Sikdara\AppData\Local\Temp\__autograph_generated_file_a_2w_4x.py", line 11, in tf__call
        ag__.